# Курсовая работа по курсу "Введение в обработку естественного языка"

## План работы

[0. Описание возможностей и методов работы телеграмм бота](#section_0)

[1. Настройки для работы dialogflow](#section_1)

[2. Начало работы (обработка команды /start)](#section_2)

[3. Функция получения прогноза погоды](#section_3)

[4. Функция болталки](#section_4)

[5. Функция получения темы курсовой работы](#section_5)

[6. Функция анализа отзыва о работе телеграмм бота](#section_6)

[7. Хендлеры](#section_7)

### 0. Описание возможностей и методов работы телеграмм бота <a id='section_0'></a>

Настоящий бот под именем **my_bot** можно найти в телеграмме.

![title](bot.jpg)

Работа с телеграмм ботом начитается с команды **/start**, которая покажет все его возможности. В качестве ответа будет предложено выбрать одно из возможных действий. Обработку запросов от inline клавиатуры см. в разделе [2. Начало работы (обработка команды /start)](#section_2).

![title](start.jpg) 

При нажатии на кнопку **"Получить тему курсовой работы по теории групп"**, последует форма ввода примерной темы. В качестве ответа будет предложена тема из базы данных, наиболее близкая к введенному запросу. В основе этой функции телеграмм бота лежит модель Word2Vec с поиском ближайших соседей. Код обработки этой функции см. в разделе [5. Функция получения темы курсовой работы](#section_5). Отметим, что чтобы воспользоваться данной функцией, не обязательно начинать с команды /start и выводить клавиатуру, можно просто написать: "Курсовая: *примерная тема*".

![title](theme.jpg)

В ответ на нажатие на кнопки **"Архив с книгами курсовой работы"** будет выслан файл books.rar. Обработка этой функции в разделе  [2. Начало работы (обработка команды /start)](#section_2).

![title](books.jpg) 

При выборе функции **"Поговорить"** бот перейдет в режим болталки (на основе dialogflow). Код этой функции см. в разделе [4. Функция болталки](#section_4). Просто поговорить с ботом можно и не выбирая команду с клавиатуру, а написав любое сообщение в чат (запросы с темой курсовой работы, отзывом и погодой будут отфильтрованы).

![title](talk.jpg) 

При нажатии на кнопку **"Оставить отзыв"** будет предложена форма ввода отзыва. Введенный отзыв классифицируется на положительный и нейтральный предобученной Bert-моделью SkolkovoInstitute/russian_toxicity_classifier. Чтобы оставить отзыв, не вызывая клавиатуру, необходимо ввести: "Отзыв: *текст отзыва*". Код функции см. в разделе [6. Функция анализа отзыва о работе телеграмм бота](#section_6). Заметим, что выполнение этой команды заметно более длительное по сравнению с другими функциями бота.

![title](review.jpg) 

Функция **"Прогноз погоды"** возвращает погоду в введенном городе на основе запроса: "Город: *свой город*" (как из клавиатуры, так и без нее). В основе функции лежит geopy и yaweather (на русском языке!). Полный код см. в [3. Функция получения прогноза погоды](#section_3). При создании этой функции потребовалось получение api-key от Погоды яндекса.

![title](weather.jpg) 

При нажатии на кнопку **"GitHub автора"** клавиатуры будет выведена ссылка на GitHub (где, в частности, в https://github.com/EkaterinaBazhanova/NLP/blob/main/course/bot_code.ipynb и находится полный код данного телеграмм бота). Обработка данный функции описана в разделе [2. Начало работы (обработка команды /start)](#section_2).

![title](link.jpg) 

### 1. Настройки для работы dialogflow <a id='section_0'></a>

In [1]:
#импорт необходимых библиотек и модулей
import os
import logging
import telegram
import re
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, CallbackQueryHandler
from google.cloud import dialogflow

#настройки dialogflow
updater = Updater("5301400304:AAH2GLKvC4vpcHNRKvZ_NtXpXab1gvfECug", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'bazhanovaen-bot-jbgt-84e6267adf6d.json'# скачaнный JSON


DIALOGFLOW_PROJECT_ID = 'bazhanovaen-bot-jbgt' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'my_bot'  # ID бота из телеграмма

### 2. Начало работы (обработка команды /start) <a id='section_2'></a>

In [2]:
#обработка команды старт (создаем Inline клавиатуру)
def startCommand(update: Update, context: CallbackContext):
    buttonA = telegram.InlineKeyboardButton('Получить тему курсовой работы по теории групп', callback_data='buttomA')
    buttonAa = telegram.InlineKeyboardButton('Архив с книгами для курсовой работы', callback_data='buttomAa')
    buttonB = telegram.InlineKeyboardButton('Поговорить', callback_data='buttomB')
    buttonC = telegram.InlineKeyboardButton('Оставить отзыв', callback_data='buttomC')
    buttonE = telegram.InlineKeyboardButton('Прогноз погоды', callback_data='buttomE')
    buttonD = telegram.InlineKeyboardButton('GitHub автора', url='https://github.com/EkaterinaBazhanova')
    markup = telegram.InlineKeyboardMarkup(inline_keyboard = [[buttonA], [buttonAa], [buttonB], [buttonC], [buttonE], [buttonD]])
    
    update.message.reply_text('Добрый день! Чтобы начать работу, выберите одно из возможных действий', 
                              reply_markup = markup)
    return callback
    
#обработка нажатия клавиш клавиатуры
def callback(update: Update, context: CallbackContext):
    query = update.callback_query
    variant = query.data
    if variant == 'buttomA':
        query.answer()
        query.edit_message_text(text='О чем хотите писать курсовую? (форма ответа: "Курсовая: примерная тема")')
    
    if variant == 'buttomAa':
#         chat_id = 1384328181
#         updater.bot.send_document(chat_id = chat_id, document = open('books.rar', 'rb'))
        chat_id=update.effective_chat.id
        updater.bot.send_document(chat_id = chat_id, document = open('books.rar', 'rb'))
    if variant == 'buttomB':
        query.answer()
        query.edit_message_text('Давайте поговорим')
    
    if variant == 'buttomC':
        query.answer()
        query.edit_message_text(text='Напишите свой отзыв о работе бота (форма ответа: "Отзыв: текст отзыва")')
    
    if variant == 'buttomE':
        query.answer()
        query.edit_message_text(text='Напишите: "Город: свой город"')

### 3. Функция получения прогноза погоды <a id='section_3'></a>

In [3]:
#импорт необходимых библиотек и модулей
#!pip install yaweather
from geopy import geocoders
from yaweather import Russia, YaWeather

#функция получения погоды
def get_weather(city: str):
    geolocator = geocoders.Nominatim(user_agent="telebot")
    latitude = geolocator.geocode(city).latitude
    longitude = geolocator.geocode(city).longitude
    y = YaWeather(api_key='f5fc7043-3196-49ed-9848-02c19a3ba8e4')
    res = y.forecast((latitude, longitude))
    forecast = f'cейчас {res.fact.temp} °C, oщущается как {res.fact.feels_like} °C'
    return forecast

#печать прогноза погоды
def print_weather(update: Update, context: CallbackContext):
    city = update.message.text
    forecast = get_weather(re.sub(r'Город: ','',city))
    update.message.reply_text(f'Погода в городе {city[7:]}: {forecast}', parse_mode='Markdown')

### 4. Функция болталки <a id='section_4'></a>

In [4]:
#болталка
def detect_intent_texts(project_id, session_id, text, language_code):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(project_id, session_id)
    
    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)

    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
        
    return response.query_result.fulfillment_text

def textMessage(update: Update, context: CallbackContext):
  
    text_user = update.message.text
       
    text_bot = detect_intent_texts(DIALOGFLOW_PROJECT_ID, SESSION_ID, text_user, DIALOGFLOW_LANGUAGE_CODE)
    if text_bot:
        update.message.reply_text(text_bot)
    else:
        update.message.reply_text('Что?')

### 5. Функция получения темы курсовой работы <a id='section_5'></a>

In [5]:
#импорт необходимых библиотек и модулей
import pandas as pd
import string
import annoy
import pickle
import numpy as np
import tqdm 
from tqdm import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

#препроцессинг текста
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

#0. датасет с темами
df = pd.read_excel('theme.xlsx')

#1. Функция преобразования темы (лемматизация, удаление стоп-слов и пунктуации)
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

#2. Процедура подготовки данных к обучению (создание списка токенов для каждой темы)
assert True
tweet_tokens = []

for text in df['Theme']:
    tweet_prep = preprocess_txt(text)
    tweet_tokens.append(tweet_prep)
    
#3. Функция поиска 5 ближайших тем
def get_response_theme(tweet, index, model, index_map):
    tweet_token = preprocess_txt(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet_token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_tweet = index.get_nns_by_vector(vector, 5)
    
    result = [index_map[i] for i in similar_tweet]
    return result

#4. Процедура поиска ближайшей темы
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for tweet in df['Theme']:
        n_model = 0
        index_map[counter] = tweet
        tweet_prep = preprocess_txt(tweet)
        
        vector_model = np.zeros(300) #вектор твита

        for word in tweet_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1
        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор твита
       
        model_index.add_item(counter, vector_model) #добавляем элемент для индексации
     
        counter += 1

        if counter > 100000:
            break

    model_index.build(10) #из индексов создаем лес из 10 деревьев
    return model_index, index_map

modelW2V = Word2Vec(sentences=tweet_tokens, vector_size=300, window=5, min_count=1)

w2v_index, index_map = get_model_index(modelW2V.wv)

#вывод темы курсовой работы
def text_theme(update: Update, context: CallbackContext):
    txt_q = update.message.text # текст который пришёл
    answer = get_response_theme(re.sub(r'Курсовая: ','',txt_q), w2v_index, modelW2V.wv, index_map)
    update.message.reply_text(f'Ваша тема: {answer[0]}', parse_mode='Markdown')

### 6. Функция анализа отзыва о работе телеграмм бота <a id='section_6'></a>

In [6]:
#импорт библиотек и модулей
from transformers import BertTokenizer, BertForSequenceClassification

def get_response_review(review):
    #создание экземляра модели и токенизатора
    tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
    model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
    
    #подготовка входа
    batch = tokenizer.encode(review, return_tensors='pt')

    if model(batch)['logits'][0].argmax().numpy() == 0:
        return 'Спасибо за Ваш положительный отзыв!'
    if model(batch)['logits'][0].argmax().numpy() == 1:
        return 'Спасибо за Ваш отзыв. Будем работать над устранением недостатков'

#функция вывода отзыва
def text_review(update: Update, context: CallbackContext):
    txt_q = update.message.text # текст который пришёл
    review_classif = get_response_review(re.sub(r'Отзыв: ','',txt_q))
    update.message.reply_text(review_classif)

### 7. Хендлеры <a id='section_7'></a>

In [7]:
# Хендлеры
start_command_handler = CommandHandler('start', startCommand)
theme_handler = MessageHandler(Filters.regex('Курсовая: '), text_theme)
review_handler = MessageHandler(Filters.regex('Отзыв: '), text_review)
weather_handler = MessageHandler(Filters.regex('Город: '), print_weather)
text_message_handler = MessageHandler(~Filters.regex('Курсовая: ') & ~Filters.regex('Отзыв: ') & ~Filters.regex('Город: '), textMessage)
callback_buttom_handler = CallbackQueryHandler(callback = callback, pattern=None, run_async=False)
                                               
# Добавляем хендлеры в диспетчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(callback_buttom_handler)
dispatcher.add_handler(theme_handler)
dispatcher.add_handler(weather_handler)
dispatcher.add_handler(review_handler)
dispatcher.add_handler(text_message_handler)

# Начинаем поиск обновлений
updater.start_polling()
# Останавливаем бота, если были нажаты Ctrl + C
updater.idle()